# DAT405/DIT407 Introduction to Data Science and AI 
## 2022-2023, Reading Period 4
## Assignment 5: Reinforcement learning and classification



Assignment 5 by Josefin Kokkinakis and Eli Uhlin, group 30.
We have both worked around 15 hours each.

## Question 1

The first question covers a deterministic MPD, where the action is directly given by the state, described as follows:

* The agent starts in state **S** (see table below)
* The actions possible are **N** (north), **S** (south), **E** (east), and **W** west. 
* The transition probabilities in each box are deterministic (for example P(s'|s,N)=1 if s' north of s). Note, however, that you cannot move outside the grid, thus all actions are not available in every box.
* When reaching **F**, the game ends (absorbing state).
* The numbers in the boxes represent the rewards you receive when moving into that box. 
* Assume no discount in this model: $\gamma = 1$
    
    
| | | |
|----------|----------|---------|
|-1 |1|**F**|
|0|-1|1|  
|-1 |0|-1|  
|**S**|-1|1|

Let $(x,y)$ denote the position in the grid, such that $S=(0,0)$ and $F=(2,3)$.


**1a)** What is the optimal path of the MDP above? Is it unique? Submit the path as a single string of directions. E.g. NESW will make a circle.

Answer 1a:
EENN which is unique.


**1b)** What is the optimal policy (i.e. the optimal action in each state)? It is helpful if you draw the arrows/letters in the grid.

Answer 1b:
in (0,0) N or E
in (0,1) N or E
in (0,2) N or E or S
in (0,3) E
in (1,0) E
in (1,1) N or S or E or W
in (1,2) N or E
in (1,3) E
in (2,0) N or W
in (2,1) N or S
in (2,2) N


**1c)** What is expected total reward for the policy in 1a)?

Answer 1c: 0 

    since 
      (0,0) -> (1,0) means -1

      (1,0) -> (2,0) means 1, and -1+1=0 total
      
      (2,0) -> (2,1) means -1, and -1+0=-1 total
      
      (2,1) -> (2,2) means +1, and -1+1=0 total
      
      (2,2) -> F     means that the total reward is 0.

## Value Iteration

For larger problems we need to utilize algorithms to determine the optimal policy $\pi^*$. *Value iteration* is one such algorithm that iteratively computes the value for each state. Recall that for a policy to be optimal, it must satisfy the Bellman equation above, meaning that plugging in a given candidate $V^*$ in the right-hand side (RHS) of the Bellman equation should result in the same $V^*$ on the left-hand side (LHS). This property will form the basis of our algorithm. Essentially, it can be shown that repeated application of the RHS to any intial value function $V^0(s)$ will eventually lead to the value $V$ which statifies the Bellman equation. Hence repeated application of the Bellman equation will also lead to the optimal value function. We can then extract the optimal policy by simply noting what actions that satisfy the equation.    

The process of repeated application of the Bellman equation is what we here call the _value iteration_ algorithm. It practically procedes as follows:

```
epsilon is a small value, threshold
for x from i to infinity 
do
    for each state s
    do
        V_k[s] = max_a Σ_s' p(s′|s,a)*(r(a,s,s′) + γ*V_k−1[s′])
    end
    if  |V_k[s]-V_k-1[s]| < epsilon for all s
        for each state s,
        do
            π(s)=argmax_a ∑_s′ p(s′|s,a)*(r(a,s,s′) + γ*V_k−1[s′])
            return π, V_k 
        end
end

```

**Example:** We will illustrate the value iteration algorithm by going through two iterations. Below is a 3x3 grid with the rewards given in each state. Assume now that given a certain state $s$ and action $a$, there is a probability 0.8 that that action will be performed and a probability 0.2 that no action is taken. For instance, if we take action **E** in state $(x,y)$ we will go to $(x+1,y)$ 80 percent of the time (given that that action is available in that state), and remain still 20 percent of the time. We will use have a discount factor $\gamma = 0.9$. Let the initial value be $V^0(s)=0$ for all states $s\in S$. 

**Reward**:

| | | |  
|----------|----------|---------|  
|0|0|0|
|0|10|0|  
|0|0|0|  


**Iteration 1**: The first iteration is trivial, $V^1(s)$ becomes the $\max_a \sum_{s'} p(s'|s,a) r(s,a,s')$ since $V^0$ was zero for all $s'$. The updated values for each state become

| | | |  
|----------|----------|---------|  
|0|8|0|
|8|2|8|  
|0|8|0|  
  
**Iteration 2**:  
  
Staring with cell (0,0) (lower left corner): We find the expected value of each move:  
Action **S**: 0  
Action **E**: 0.8( 0 + 0.9 \* 8) + 0.2(0 + 0.9 \* 0) = 5.76  
Action **N**: 0.8( 0 + 0.9 \* 8) + 0.2(0 + 0.9 \* 0) = 5.76  
Action **W**: 0

Hence any action between **E** and **N** would be best at this stage.

Similarly for cell (1,0):

Action **N**: 0.8( 10 + 0.9 \* 2) + 0.2(0 + 0.9 \* 8) = 10.88 (Action **N** is the maximizing action)  

Similar calculations for remaining cells give us:

| | | |  
|----------|----------|---------|  
|5.76|10.88|5.76|
|10.88|8.12|10.88|  
|5.76|10.88|5.76|  



## Question 2

**2a)** Code the value iteration algorithm just described here, and show the converging optimal value function and the optimal policy for the above 3x3 grid. Make sure to consider that there may be several equally good actions for a state when presenting the optimal policy.

Steps:
1. Define matrix
2. Define grid?
3. function that calculates what action to take in order to maximize the reward

value of s= max_expected(possible_actions)

expected_value(prob_of_moving*sum_of_reward_for_transitioning + discounted_value)


moving_prob=0.8
functionen V_k(x,y,V_prev, reward, moving_prob, gamma=1)

In [3]:
# Answer 2a
from random import shuffle
from enum import Enum
from math import sqrt
epsilon = 0.1
gamma = 0.9
delta = 0.2
from typing import List

class State:
    def __init__(self ,val, coord):
        self.val = val

        self.actions = []
        self.coord = coord

    def action_list_to_string(self):
        newlst = []
        for a in self.actions:
            newlst.append(str(a.coord))

        return ' '.join(newlst)


    def __str__(self):
        return "(" + str(self.get_x()) + ", " + str(self.get_y()) + ") has value: "+ str(self.val) + " and actions: " + str(self.action_list_to_string())

    def get_coord(self):
        return self.coord

    def get_x(self):
        return self.coord[0]

    def get_y(self):
        return self.coord[1]

    def get_val(self):
        return self.val

    def set_val(self, new_value):
        self.val = new_value

    def get_actions(self):
        return self.actions

    def add_action(self, state):
        self.actions.append(state)

    def get_max_action(self):
        greatest_state: State = None
        for a in self.actions:
            if greatest_state is None:
                greatest_state = a
            elif a.get_val() > greatest_state.get_val():
                greatest_state = a
        return greatest_state

    def find_action(self, coord):
        for a in self.actions:
            if a.get_coord() == coord:
                return a


def listcoord(rows, cols):
    return [(x ,y) for x in range(rows) for y in range(cols)]


def State_list(listval, listcoord):
    state_list = []
    for v, c in zip(listval, listcoord):
        state_list.append(State(v ,c))
    for s in state_list:
        for t in state_list:
            abs_x = abs(s.get_x() - t.get_x())
            abs_y = abs(s.get_y() - t.get_y())
            if ((abs_y == 1 and abs_x == 0) ^ (abs_y == 0 and abs_x == 1)):
                s.add_action(t)
    return state_list

#0.8( 10 + 0.9 * 2) + 0.2(0 + 0.9 * 8)
#Action N: 0.8( 10 + 0.9 * 2) + 0.2(0 + 0.9 * 8) = 10.88

def value_function(s: State, o: State, generation):
    s_val = s.get_val()
    o_val = o.get_val()
    action = s.get_max_action()
    action_val = action.get_val()
    old_action_val = o.find_action(action.get_coord()).get_val()
    return (1-delta)*(old_action_val + (gamma**generation)*action_val) + delta*(o_val + (gamma**generation)*s_val)

#0.8*(0+10)+0.2*(0+0.9*0)
#0.8*(nuvarande_värde action + gamma* förra värde action) + 0.2(nuvarande + gamma*värdet_om_stå_kvar)


#
#TODO add new generation values to new list so that they don't effect other states during current gen.

def new_Generation(state: List[State], old_state, generation):
    next_gen_val = []
    next_gen_coord = []
    for s,o in zip(state, old_state):
        next_gen_val.append(value_function(s, o, generation))
        next_gen_coord.append(s.get_coord())
    generation += 1
    old_state = state
    state = State_list(next_gen_val, next_gen_coord)
    return (state, old_state, generation)

def check_epsilon(state: List[State], old_state):
    for s, o in zip(state, old_state):
        if abs(s.get_val() - o.get_val()) >= epsilon:
            return False
    return True

def run_til_convergence(statelist: List[State], oldlist):
    generation = 0
    while not check_epsilon(statelist, oldlist):
        (statelist, oldlist, generation) = new_Generation(statelist, oldlist, generation)
        print("Generation: ", generation)
        print("Gamma**generation: ", gamma ** generation)
        print_matrix(statelist)
    print("final generation")


def print_matrix(state: List[State]):
    dimension = int(sqrt(len(state)))
    mat = [[0 for i in range(dimension)] for i in range(dimension)]
    for s in state:
        mat[s.get_x()][s.get_y()] = s.get_val()
    for row in mat:
        print(row)


coord = listcoord(3,3)
vals = [0 for x in range(9)]
oldlist = State_list(vals, coord)
vals[4] = 10
statelist = State_list(vals, coord)
generation = 0

run_til_convergence(statelist, oldlist)

                           


            

Generation:  1
Gamma**generation:  0.9
[0.0, 8.0, 0.0]
[8.0, 2.0, 8.0]
[0.0, 8.0, 0.0]
Generation:  2
Gamma**generation:  0.81
[5.760000000000001, 10.88, 5.760000000000001]
[10.88, 8.120000000000001, 10.88]
[5.760000000000001, 10.88, 5.760000000000001]
Generation:  3
Gamma**generation:  0.7290000000000001
[14.383360000000003, 10.224320000000002, 14.383360000000003]
[10.224320000000002, 15.165680000000004, 10.224320000000002]
[14.383360000000003, 10.224320000000002, 14.383360000000003]
Generation:  4
Gamma**generation:  0.6561
[17.915917312000005, 19.007330432000007, 17.915917312000005]
[19.007330432000007, 18.501979568000007, 19.007330432000007]
[17.915917312000005, 19.007330432000007, 17.915917312000005]
Generation:  5
Gamma**generation:  0.5904900000000001
[23.38362226682881, 26.382868934938887, 23.38362226682881]
[26.382868934938887, 23.61698935606113, 26.382868934938887]
[23.38362226682881, 26.382868934938887, 23.38362226682881]
Generation:  6
Gamma**generation:  0.531441
[34.01366

**2b)** Explain why the result of 2a) does not depend on the initial value $V_0$.

Answer 2b:
Because of gamma. During the first iteration when generation=0, we have gamma^0, which will be equal to 1. During the following iterations it will however decrease since gamma is less than 1. As gamma decreases with each iteration, so does the proportional value of each new iteration.

**2c)** Describe your interpretation of the discount factor $\gamma$. What would happen in the two extreme cases $\gamma = 0$ and $\gamma = 1$? Given some MDP, what would be important things to consider when deciding on which value of $\gamma$ to use?

Answer 2c:
The larger our discount factor gamma is, the smaller the future discounts are. If gamma is really small we get larger future discounts. So when gamma aproaches 0, future rewards will be very small, which creates a bias towards earlier iterations and when gamma equal 1, the future rewards are just as important as the intial reward.

Chosing gamma values does therefore depends on how important furture rewards are to the learning agent. So it depends on if the agens want 

## Question 4

**4a)** What is the importance of exploration in reinforcement learning? Explain with an example.

Answer 4a:
Example: Imagine a person that is going to invest their money in a fund. If they find exploring to be very important and they don't aim for short term rewards, they aim for long term rewards. They will not just go for the first aalternative which at first glance may seem good but investigating more options gives the investor a greater understanding of how good the investment in the first fund is, in comparison to other funds.

**4b)** Explain what makes reinforcement learning different from supervised learning tasks such as regression or classification. 


Answer 4b:
Reinforcement learning is different from supervised learning since it does not require labeled data. It learns the data by exploring the environment. 

## Question 5

**5a)** Give a summary of how a decision tree works and how it extends to random forests.

Answer 5a:
When using decision trees, we classify data by splitting it up depending on how it "responds" to a certain question. For example, lets say we want to predict how likely it is for us to "go for a run" on a monday morning. Factors that may inpact my decision about going for a run could be, answers to the questions: is it raining?, did I sleep well?, etc. The tree contains questions like these and splits the input space into smaller regions giving us a more complex and deeper tree that better fits our data.
A disadvantage with using decision trees is that they are prone to overfitting since they are not "flexible" when it comes to classifying new samples. So when trees grow deeper, relationships between certain features may not be clear. This is why random forests was created. Random forests combines the predicitions of several decision trees. 

**5b)** State at least one advantage and one drawback with using random forests over decision trees.

Answer 5b: An advantage with using random forests over decision trees is that they reduce overfitting. By combining the predictions of several decision trees they also have the flexability that tends to result in higher accuracy. 
A drawback with using random forests over decision trees is that random forests are more expensive since they contain several decision trees. Thus when making a prediction, all decision trees have to make predictions for the same given input.
It is also more difficult to interpret than decision trees. Decision trees can be interpreted easily just by following the path in the tree that leads to the leaf.